# Doladění modelů Open AI

Tento sešit je založen na aktuálních pokynech uvedených v dokumentaci [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) od Open AI.

Doladění zlepšuje výkon základních modelů pro vaši aplikaci tím, že je znovu trénuje s dalšími daty a kontextem relevantním pro konkrétní použití nebo scénář. Všimněte si, že techniky návrhu promptů jako _few shot learning_ a _retrieval augmented generation_ vám umožňují vylepšit výchozí prompt o relevantní data pro zlepšení kvality. Tyto přístupy jsou však omezeny maximální velikostí tokenového okna cílového základního modelu.

S doladěním efektivně znovu trénujeme samotný model s požadovanými daty (což nám umožňuje použít mnohem více příkladů, než kolik se vejde do maximálního tokenového okna) - a nasazujeme _vlastní_ verzi modelu, která již nemusí mít při inferenci poskytované příklady. To nejen zlepšuje efektivitu našeho návrhu promptu (máme větší flexibilitu v používání tokenového okna pro jiné věci), ale potenciálně také zlepšuje naše náklady (snížením počtu tokenů, které musíme posílat modelu při inferenci).

Doladění má 4 kroky:
1. Připravit tréninková data a nahrát je.
1. Spustit tréninkový úkol pro získání doladěného modelu.
1. Vyhodnotit doladěný model a iterovat pro kvalitu.
1. Nasadit doladěný model pro inferenci, když jste spokojeni.

Všimněte si, že ne všechny základní modely podporují doladění - [zkontrolujte dokumentaci OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) pro nejnovější informace. Můžete také doladit již dříve doladěný model. V tomto tutoriálu použijeme `gpt-35-turbo` jako náš cílový základní model pro doladění.

---


### Krok 1.1: Připravte svůj dataset

Postavme chatbot, který vám pomůže pochopit periodickou tabulku prvků tím, že bude odpovídat na otázky o prvku pomocí limeriku. V _tomto_ jednoduchém tutoriálu vytvoříme pouze dataset pro trénink modelu s několika ukázkovými příklady odpovědí, které ukazují očekávaný formát dat. V reálném použití byste potřebovali vytvořit dataset s mnohem více příklady. Můžete také použít otevřený dataset (pro vaši aplikační oblast), pokud existuje, a přeformátovat ho pro použití při doladění.

Protože se zaměřujeme na `gpt-35-turbo` a hledáme odpověď v jednom kole (chat completion), můžeme vytvořit příklady pomocí [tohoto navrhovaného formátu](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), který odráží požadavky OpenAI na chat completion. Pokud očekáváte vícekrokový konverzační obsah, použili byste [formát vícekrokových příkladů](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), který zahrnuje parametr `weight` pro signalizaci, které zprávy by měly být použity (nebo ne) při doladění.

Pro náš tutoriál použijeme jednodušší formát jednoho kola. Data jsou ve formátu [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) s jedním záznamem na řádek, každý reprezentovaný jako JSON objekt. Níže uvedený úryvek ukazuje 2 záznamy jako ukázku – viz [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) pro kompletní ukázkovou sadu (10 příkladů), kterou použijeme pro náš tutoriál doladění. **Poznámka:** Každý záznam _musí_ být definován v jednom řádku (nesmí být rozdělen přes více řádků, jak je obvyklé ve formátovaném JSON souboru).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

V reálném použití budete potřebovat mnohem větší sadu příkladů pro dobré výsledky – kompromis bude mezi kvalitou odpovědí a časem/náklady na doladění. Používáme malou sadu, abychom mohli doladění rychle dokončit a ilustrovat proces. Viz [tento příklad z OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) pro složitější tutoriál doladění.


---

### Krok 1.2 Nahrajte svůj dataset

Nahrajte data pomocí Files API [jak je popsáno zde](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Všimněte si, že abyste mohli tento kód spustit, musíte nejprve provést následující kroky:
 - Nainstalovat Python balíček `openai` (ujistěte se, že používáte verzi >=0.28.0 pro nejnovější funkce)
 - Nastavit proměnnou prostředí `OPENAI_API_KEY` na svůj OpenAI API klíč
Pro více informací si přečtěte [Průvodce nastavením](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) poskytnutý pro kurz.

Nyní spusťte kód pro vytvoření souboru k nahrání z vašeho lokálního JSONL souboru.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Krok 2.1: Vytvoření úlohy doladění pomocí SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Krok 2.2: Zkontrolujte stav úlohy

Zde je několik věcí, které můžete dělat s API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Vyjmenuje posledních n úloh jemného doladění
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Získá podrobnosti o konkrétní úloze jemného doladění
- `client.fine_tuning.jobs.cancel(<job_id>)` - Zruší úlohu jemného doladění
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Vyjmenuje až n událostí z úlohy
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvním krokem procesu je _ověření tréninkového souboru_, aby bylo zajištěno, že data jsou ve správném formátu.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Krok 2.3: Sledujte události pro monitorování pokroku


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Krok 2.4: Zobrazit stav v OpenAI Dashboard


Stav můžete také zobrazit návštěvou webu OpenAI a prozkoumáním sekce _Fine-tuning_ na platformě. Zobrazí se vám stav aktuální úlohy a také možnost sledovat historii předchozích běhů úloh. Na tomto snímku obrazovky vidíte, že předchozí běh selhal a druhý běh byl úspěšný. Pro kontext, k tomu došlo, když první běh použil JSON soubor s nesprávně formátovanými záznamy – po opravě druhý běh úspěšně dokončil a model byl zpřístupněn k použití.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba.cs.png)


Statusové zprávy a metriky můžete také zobrazit posunutím dolů ve vizuálním panelu, jak je znázorněno:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b.cs.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a65229.cs.png)|


---

### Krok 3.1: Získání ID a testování doladěného modelu v kódu


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Krok 3.2: Načtení a testování doladěného modelu v Playgroundu

Nyní můžete doladěný model otestovat dvěma způsoby. Nejprve můžete navštívit Playground a pomocí rozbalovacího seznamu Models vybrat svůj nově doladěný model z uvedených možností. Druhou možností je použít volbu "Playground" zobrazenou v panelu Fine-tuning (viz screenshot výše), která spustí následující _porovnávací_ zobrazení, jež ukazuje základní a doladěné verze modelu vedle sebe pro rychlé vyhodnocení.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016.cs.png)

Jednoduše vyplňte systémový kontext použitý ve vašich tréninkových datech a zadejte svou testovací otázku. Všimnete si, že obě strany jsou aktualizovány se stejným kontextem a otázkou. Spusťte porovnání a uvidíte rozdíl ve výstupech mezi nimi. _Všimněte si, jak doladěný model vykresluje odpověď ve formátu, který jste poskytli ve svých příkladech, zatímco základní model jednoduše následuje systémový prompt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350.cs.png)

Všimnete si, že porovnání také poskytuje počet tokenů pro každý model a čas potřebný pro inferenci. **Tento konkrétní příklad je zjednodušený a slouží k ukázce procesu, ale ve skutečnosti neodráží reálný dataset nebo scénář**. Můžete si všimnout, že oba vzorky ukazují stejný počet tokenů (systémový kontext a uživatelský prompt jsou identické), přičemž doladěný model potřebuje více času na inferenci (vlastní model).

V reálných scénářích nebudete používat takovýto jednoduchý příklad, ale doladění na základě reálných dat (např. katalog produktů pro zákaznický servis), kde bude kvalita odpovědi mnohem zřetelnější. V _tomto_ kontextu bude dosažení ekvivalentní kvality odpovědi se základním modelem vyžadovat více vlastního prompt engineeringu, což zvýší využití tokenů a potenciálně i související dobu zpracování inferencí. _Pro vyzkoušení si prohlédněte příklady doladění v OpenAI Cookbooku, kde můžete začít._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Prohlášení o vyloučení odpovědnosti**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože usilujeme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho mateřském jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Nejsme odpovědní za jakékoliv nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
